In [1]:
import json
import os
from PIL import Image
import numpy as np
import glob






In [ ]:
from PIL import Image
import numpy as np

def concat_images(image_paths):
    """
    이미지들을 2x2, 2x4 그리드로 병합하는 함수
    
    Args:
        image_paths (list): 이미지 파일의 경로 리스트
        output_path (str): 저장할 결과 이미지 경로
    """
    if len(image_paths) == 4:
        grid = 2
    elif len(image_paths) == 8:
        grid = 4
    else:
        raise ValueError('이미지 파일은 4개 또는 8개여야 합니다.')
    
    # 모든 이미지 로드
    images = [Image.open(path) for path in image_paths]
    
    # 모든 이미지의 크기를 첫 번째 이미지와 동일하게 맞춤
    target_size = images[0].size

    resized_images = [img.resize(target_size) for img in images]
    
    # 결과 이미지의 크기 계산
    total_width = target_size[0] * 2
    total_height = target_size[1] * grid
    
    # 새로운 빈 이미지 생성
    result = Image.new('RGB', (total_width, total_height))
    
    # 이미지들을 2xgrid 그리드로 배치
    for idx, img in enumerate(resized_images):
        x = (idx % 2) * target_size[0]
        y = (idx // 2) * target_size[1]
        result.paste(img, (x, y))
    
    # 결과 이미지 저장

    return result



In [ ]:
regions = [ 'chinese_receipt', 'japanese_receipt','thai_receipt', 'vietnamese_receipt']

# 사용 예시
for region in regions:
    ufo_path = "/data/ephemeral/home/data/"+region+"/ufo/train.json"
    output_path = "/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/SR_data/"+region

    with open(ufo_path, 'r', encoding='utf-8') as f:
        ufo_data = json.load(f)
    

    img_path_x4 = ('/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/results/'+region+'_x4/visualization/Test/')
    img_path_x2 = ('/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/results/'+region+'_x2/visualization/Test/')
    image_paths_x4 = []
    image_paths_x2 = []
    img_paths = []

    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_paths_x4.extend(glob.glob(os.path.join(img_path_x4, ext)))
        image_paths_x2.extend(glob.glob(os.path.join(img_path_x2, ext)))

    sorted_img_path_x4 = sorted(image_paths_x4)
    sorted_img_path_x2 = sorted(image_paths_x2)
    # scale 4 이미지 concat
    for i in range(len(image_paths_x4)//4):
        image_paths = sorted_img_path_x4[i*4:i*4+4]

        result = concat_images(image_paths)
        # 파일 이름만 추출 (경로 제외)
        filename = image_paths[0].split('/')[-1]

        # 파일 이름에서 base_name 추출
        output_name = filename.split('_4_')[0]
        if output_name == 'extractor.th.in_house.appen_000110_page0001' or output_name == 'extractor.th.in_house.appen_000184_page0001':
            output_name = output_name + '.png'
        else:
            output_name = output_name + '.jpg'

        # UFO 데이터에서 이미지 크기 추출
        ufo_width = ufo_data['images'][output_name]["img_w"]
        ufo_height = ufo_data['images'][output_name]["img_h"]
        if result.size[0] != ufo_width*4 or result.size[1] != ufo_height*4:
            print(f"Error: {output_name}의 크기가 맞지 않습니다. img, ufo", result.size, ufo_width*4, ufo_height*4)
            result = result.resize((ufo_width*4, ufo_height*4))
        result.save(output_path+'/'+output_name, 'JPEG', quality=95)


    # scale 2 이미지 concat
    for i in range(len(image_paths_x2)//8):
        
        image_paths = sorted_img_path_x2[i*8:(i+1)*8]

        result = concat_images(image_paths)
        # 파일 이름만 추출 (경로 제외)
        filename = image_paths[0].split('/')[-1]

        # 파일 이름에서 base_name 추출
        output_name = filename.split('_8_')[0]
        output_name = output_name + '.jpg'

        # UFO 데이터에서 이미지 크기 추출
        ufo_width = ufo_data['images'][output_name]["img_w"]
        ufo_height = ufo_data['images'][output_name]["img_h"]
        if result.size[0] != ufo_width*2 or result.size[1] != ufo_height*2:
            print(f"Error: {output_name}의 크기가 맞지 않습니다. img, ufo", result.size, ufo_width*2, ufo_height*2)
            result = result.resize((ufo_width*2, ufo_height*2))
        result.save(output_path+'/'+output_name, 'JPEG', quality=95)
        print(output_path+'/'+output_name)
        


/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/SR_data/chinese_receipt/extractor.zh.in_house.appen2_001047_page0001.jpg
Error: extractor.zh.in_house.appen2_001094_page0001.jpg의 크기가 맞지 않습니다. img, ufo (4352, 6048) 4354 6048
/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/SR_data/chinese_receipt/extractor.zh.in_house.appen2_001094_page0001.jpg
/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/SR_data/chinese_receipt/extractor.zh.in_house.appen_000016_page0001.jpg
Error: extractor.zh.in_house.appen_000058_page0001.jpg의 크기가 맞지 않습니다. img, ufo (3400, 4672) 3400 4676
/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/SR_data/chinese_receipt/extractor.zh.in_house.appen_000058_page0001.jpg
Error: extractor.zh.in_house.appen_000086_page0001.jpg의 크기가 맞지 않습니다. img, ufo (3400, 4672) 3400 4676
/data/ephemeral/home/JYP/level2-cv-datacentric-cv-14/SuperResolution/SR_data/chinese_receipt/extractor.zh.in_house.appen_000086_pa

KeyboardInterrupt: 